In [1]:
import pandas as pd
from tqdm import tqdm
import plotly.express as px

2022-11-25 12:17:54.529137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Get Wordlists

In [3]:
def process_wordlist(Pfad):
    wordlist_df = pd.read_csv(Pfad, sep=";")
    words = list(wordlist_df["Word"])
    return words

In [4]:
konkrete_wortfelder = ["Ornamente", "Dekorationsgegenstaende", "Bauwerke", "Gebaeudeteile",  "Zimmer",\
                       "Moebel", "Heimtextilien", "Haushaltsgegenstaende", "Kleidung", "Kleidungsteile" ,\
                       "Aufmachung", "aussehensspezifisch", "Taschen", "Stoffe", "formspezifisch", \
                       "helligkeitsspezifisch", "oberflaechenspezifisch", "Koerperteile", "Farben",\
                      "Pflanzen", "Pflanzenteile", "Schmuck", "Gruenflaechen", "Bilder", "Gartenanlagen",\
                      "Waelder", "Kunstobjekte", "Wege", "Gerueche", "Muster", "Holz"] 
konkrete_woerter = []
for wortfeld in konkrete_wortfelder:
    file = "Wordlists_from_fasttext/" + wortfeld + "_from_fasttext_cleaned.csv"
    konkrete_woerter += process_wordlist(file)
    
    

In [5]:
charaktereigenschaften = process_wordlist("Wordlists_from_fasttext/Charaktereigenschaften_from_fasttext_cleaned.csv")
emotionen = process_wordlist("Wordlists_from_fasttext/Emotionen_from_fasttext_cleaned.csv")
abstrakte_woerter = charaktereigenschaften + emotionen

# Romane annotieren

In [22]:
def annotate_novel(Pfad, wordlist1, wordlist2):
    with open(Pfad) as f:
        text = f.read()
    
    doc = nlp(text, disable=["ner", "tagger", "parser", "attribute_ruler"])
        
    new_text = """<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title></title>
            </titleStmt>
            <publicationStmt>
                <p></p>
            </publicationStmt>
            <sourceDesc>
                <p></p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text>
        <body>
            <p>"""
            
    for i in range(len(doc)):
        token = doc[i]
        if token.text != "&":
            if token.text == "\n":
                new_text += "</p><p>"
            else:
                if i < len(doc)-1:
                    if doc[i+1].is_punct:
                        if token.lemma_ in wordlist1:
                            new_text += "<hi rend=\"bold\">" + token.text + "</hi>"
                        elif token.lemma_ in wordlist2:
                            new_text += "<hi rend=\"italic\">" + token.text + "</hi>"
                        else:
                            new_text += token.text
                    else:
                        if token.lemma_ in wordlist1:
                            new_text += "<hi rend=\"bold\">" + token.text + "</hi>" + " "
                        else:
                            new_text += token.text + " "
                else:
                    if token.lemma_ in wordlist1:
                        new_text += "<hi rend=\"bold\">" + token.text + "</hi>"
                    elif token.lemma_ in wordlist2:
                        new_text += "<hi rend=\"italic\">" + token.text + "</hi>"
                    else:
                        new_text += token.text
                    
    new_text += """</p>
            </body>
        </text>
    </TEI>"""
    
    
    new_filename= Pfad.split("/")[1] + ".xml"
    #new_filename= "k00100000273.xml"
    
    with open(new_filename, 'w') as f:
        f.write(new_text)


    

In [8]:
# Heimburg: Trudchens Heirat
annotate_novel("txt/k00200000983", konkrete_woerter, abstrakte_woerter)

In [11]:
# Ernst Weiß: Tiere in Ketten
annotate_novel("txt/k00200002365", woerter, abstrakt_words)

In [10]:
# Sperl: Richiza
annotate_novel("txt/k00200003156", woerter, abstrakt_words)

In [7]:
# Ganghofer: Martinsklause
annotate_novel("txt/k00200000736", woerter, abstrakt_words)

In [18]:
# Marianne Ehrmann: Amalie
annotate_novel("txt/k00100000084", konkrete_woerter, abstrakte_woerter)

In [7]:
# Schlegel: Lucinde
annotate_novel("txt/k00100000349", woerter, abstrakt_words)

In [9]:
# Mereau: Blütenalter der Empfindung
annotate_novel("txt/k00100000273", konkrete_woerter, abstrakte_woerter)

# Textstellen nach Absätzen mit den meisten konkreten Wörtern soriteren

In [10]:
def get_paragraphs(filename):
    with open(filename, "r") as f:
        text = f.read()
    text_doc = nlp(text)
    
    paragraphs_list = []
    paragraphs_list_orig = []
    paragraph = ""
    paragraph_orig = ""

    for token in text_doc:
        if token.text == "\n":
            if len(paragraph.split(" ")) > 2:
                paragraph = paragraph.strip()
                paragraph_orig = paragraph_orig.strip()
                paragraphs_list.append(paragraph)
                paragraphs_list_orig.append(paragraph_orig)
            paragraph = ""
            paragraph_orig = ""
        else:
            if not token.is_space:
                paragraph = paragraph + token.lemma_ + " "
                paragraph_orig = paragraph_orig + token.text + " "
                #paragraph += " "

    df = pd.DataFrame(paragraphs_list, columns=["paragraph_lemm"])
    df["paragraph_orig"] = ""
    for i in range(len(df)):
        df["paragraph_orig"].loc[i] = paragraphs_list_orig[i]
    
    df["anteil_konkret"] = 0
    df["anteil_abstrakt"] = 0

    df["abs_konkret"] = 0
    df["abs_abstrakt"] = 0

    for i in range(len(df)):

        paragraph = df["paragraph_lemm"].loc[i]
        #print(len(paragraph.split()))

        konkret_counter = 0
        abstrakt_counter = 0
        for wort in paragraph.split():
            if wort in konkrete_woerter:
                konkret_counter +=1
            elif wort in abstrakte_woerter:
                abstrakt_counter += 1
        df["anteil_konkret"].loc[i] = konkret_counter/len(paragraph.split())
        df["anteil_abstrakt"].loc[i] = abstrakt_counter/len(paragraph.split())
        df["abs_konkret"].loc[i] = konkret_counter
        df["abs_abstrakt"].loc[i] = abstrakt_counter

    return df



In [19]:
df_lucinde = get_paragraphs("txt/k00100000349")

In [21]:
df_sorted = df_lucinde.sort_values("anteil_konkret", ascending=False)
df_sorted.to_excel("out_sorted_paragraphs/lucinde_sorted_ant_konkr.xlsx", encoding="utf-8")